In [2]:
import os
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import numpy as np
import imghdr
from PIL import Image
import shutil
from matplotlib import pyplot as plt

import tensorflow as tf
import pandas as pd
from random import shuffle
from matplotlib.pyplot import imread, imshow, subplots, show
%matplotlib inline
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [3]:
def show(img):
    try:
        cv2.imshow('img', img)
        cv2.waitKey(0)
    except:
        print('error')
        
def safe_mkdir(folder):
    assert not os.path.exists(folder)
    try:
        os.mkdir(folder)
    except OSError:
        print ("Создать директорию %s не удалось" % folder)
        
def to_grayscale(source_folder):
#     Принимает на вход папку, содержащую подпапки с изображениями, создаёт на том же уровне папку с добавлением к названию grayscale, внутри создаёт подпапки с изображениями
    assert os.path.exists(source_folder)
    output_folder = source_folder + ' grayscale'
    safe_mkdir(output_folder)
    for folder in tqdm(os.listdir(source_folder)):
        safe_mkdir(output_folder + '\\' + folder)
        for image in os.listdir(source_folder + '\\' + folder):
            img = cv2.imread(source_folder + '\\' + folder + '\\' + image, cv2.IMREAD_GRAYSCALE)
            image_path = source_folder + '\\' + folder + '\\' + image
            print(f'image: {image_path}, img: {img}')
            cv2.imwrite(output_folder + '\\' + folder + '\\' + image, img)
            
def to_grayscale_and_resize(source_folder, width, height):
#     Принимает на вход папку, содержащую подпапки с изображениями, создаёт на том же уровне папку с добавлением к названию _grayscale_resized, внутри создаёт подпапки с изображениями, сохраняя структуру подпапок
    assert os.path.exists(source_folder)
    output_folder = source_folder + '_grayscale_resized'
    safe_mkdir(output_folder)
    for folder in tqdm(os.listdir(source_folder)):
        safe_mkdir(output_folder + '\\' + folder)
        for image in os.listdir(source_folder + '\\' + folder):
            img = cv2.imread(source_folder + '\\' + folder + '\\' + image, cv2.IMREAD_GRAYSCALE)
            img = cv2.resize(img, (width, height))
            cv2.imwrite(output_folder + '\\' + folder + '\\' + image, img)

In [77]:
# переименовываем все изображения в формат 000i.ext, ну просто для порядка
for folder in os.listdir(r'C:\Nikita\Parser\the whole 52 cards deck'):
    card_folder = 'C:\\Nikita\\Parser\\the whole 52 cards deck\\' + folder
    for i, file in enumerate(os.listdir(card_folder)):
        ext_len = file.rfind('.')
        ext = file[ext_len:]
        os.rename(card_folder + '\\' + file, card_folder + '\\' + str(i).zfill(4) + ext)

In [54]:
# проверяем, не затесались ли среди изображений гифки, а то cv2 с ними работать не умеет, трансформируем их в png
for folder in os.listdir(r'C:\Nikita\Parser\the whole 52 cards deck'):
    card_folder = 'C:\\Nikita\\Parser\\the whole 52 cards deck\\' + folder
    for file in os.listdir(card_folder):
        filetype = imghdr.what(card_folder + '\\' + file)
        if filetype == 'jpeg' or filetype == 'png':
            pass
        else:
            with Image.open(card_folder + '\\' + file) as img:
                ext_len = file.rfind('.')
                ext = file[ext_len:]
                new_filename = file[:ext_len] + '.png'
                img.save(card_folder + '\\' + new_filename)
            os.remove(card_folder + '\\' + file)
#             print(f'{card_folder}\\{file}: {filetype}')        

In [96]:
# проверяем, нет ли среди изображений файлов нулевого размера, или не являющихся изображениями вообще
for folder in os.listdir('C:\\Nikita\\Parser\\the whole 52 cards deck grayscale'):
    card_folder = 'C:\\Nikita\\Parser\\the whole 52 cards deck grayscale\\' + folder
    for file in os.listdir(card_folder):
        with Image.open(card_folder + '\\' + file) as img:
            print(f'{card_folder}\\{file}, size: {img.width}x{img.height}')
            if img.width == 0 or img.height == 0:
                print(f'0 size detected: {card_folder}\\{file}')

C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0000.jpg, size: 300x472
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0001.jpg, size: 350x540
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0002.jpg, size: 354x546
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0003.jpg, size: 370x560
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0004.jpg, size: 521x800
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0005.jpg, size: 1033x1396
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0006.jpg, size: 577x1024
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0007.jpg, size: 800x800
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0008.png, size: 700x700
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0009.png, size: 335x500
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0010.jpg, size: 667x760
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0011.jpg, size: 684x1024
C:\Nikita\Parser\the whole 52 cards deck grayscale\2C\0012.j

C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0005.jpg, size: 755x1110
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0006.png, size: 731x370
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0007.jpg, size: 1365x2048
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0008.jpg, size: 472x717
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0009.jpg, size: 709x1040
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0010.png, size: 700x700
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0011.jpg, size: 440x596
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0012.jpg, size: 900x1240
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0013.jpg, size: 425x623
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0014.jpg, size: 750x1106
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0015.png, size: 900x1024
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\0016.jpg, size: 111x150
C:\Nikita\Parser\the whole 52 cards deck grayscale\9C\001

C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0013.png, size: 800x1112
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0014.jpg, size: 511x720
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0015.jpg, size: 540x799
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0016.jpg, size: 666x1000
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0017.jpg, size: 667x1024
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0018.jpg, size: 515x800
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0019.jpg, size: 666x1000
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0020.jpg, size: 800x800
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0021.jpg, size: 802x1200
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0022.jpg, size: 666x1000
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0023.jpg, size: 602x880
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\0024.jpg, size: 649x1024
C:\Nikita\Parser\the whole 52 cards deck grayscale\QH\002

In [113]:
with Image.open('C:\\Nikita\\Parser\\the whole 52 cards deck grayscale test\\2C\\0000.jpg') as img:
#     plt.imshow(img)
    print(img.width, img.height)
type(img)

300 472


PIL.JpegImagePlugin.JpegImageFile

In [114]:
img = cv2.imread('C:\\Nikita\\Parser\\the whole 52 cards deck grayscale test\\2C\\0000.jpg')

In [116]:
img.shape

(472, 300, 3)

In [83]:
source_folder = r'C:\Nikita\Parser\the whole 52 cards deck'
to_grayscale(source_folder)

  2%|█▌                                                                                 | 1/52 [00:00<00:07,  7.15it/s]

image: C:\Nikita\Parser\the whole 52 cards deck\2C\0000.jpg, img: [[236 232 228 ... 220 223 226]
 [141 137 133 ... 123 126 129]
 [115 110 105 ... 104 107 110]
 ...
 [116 102  96 ...  96  92 104]
 [114 106 100 ...  99  99 112]
 [126 125 119 ... 111 112 125]]
image: C:\Nikita\Parser\the whole 52 cards deck\2C\0001.jpg, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
image: C:\Nikita\Parser\the whole 52 cards deck\2C\0002.jpg, img: [[56 55 54 ... 57 58 58]
 [54 55 55 ... 59 60 60]
 [54 54 56 ... 61 62 62]
 ...
 [60 56 56 ... 59 62 63]
 [55 55 74 ... 61 61 63]
 [63 54 64 ... 58 61 63]]
image: C:\Nikita\Parser\the whole 52 cards deck\2C\0003.jpg, img: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
image: C:\Nikita\Parser\the whole 52 cards deck\2C\0004.jpg, img: [[0 0 0 ..

  4%|███▏                                                                               | 2/52 [00:00<00:08,  5.88it/s]

image: C:\Nikita\Parser\the whole 52 cards deck\2D\0003.jpg, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
image: C:\Nikita\Parser\the whole 52 cards deck\2D\0004.png, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
image: C:\Nikita\Parser\the whole 52 cards deck\2D\0005.png, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
image: C:\Nikita\Parser\the whole 52 cards deck\2D\0006.jpg, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]


  6%|████▊                                                                              | 3/52 [00:00<00:08,  5.85it/s]

...

image: C:\Nikita\Parser\the whole 52 cards deck\TD\0008.jpg, img: [[252 248 245 ... 220 221 221]
 [253 254 255 ... 220 221 221]
 [246 254 255 ... 219 220 220]
 ...
 [253 253 253 ... 255 255 255]
 [253 253 253 ... 255 255 255]
 [253 253 253 ... 255 255 255]]
image: C:\Nikita\Parser\the whole 52 cards deck\TD\0009.png, img: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
image: C:\Nikita\Parser\the whole 52 cards deck\TD\0010.jpg, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
image: C:\Nikita\Parser\the whole 52 cards deck\TD\0011.png, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
image: C:\Nikita\Parser\the whole 52 car

 98%|████████████████████████████████████████████████████████████████████████████████▍ | 51/52 [00:14<00:00,  2.95it/s]

image: C:\Nikita\Parser\the whole 52 cards deck\TS\0000.jpg, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
image: C:\Nikita\Parser\the whole 52 cards deck\TS\0001.jpg, img: [[94 94 94 ... 94 94 94]
 [94 94 94 ... 94 94 94]
 [94 94 94 ... 94 94 94]
 ...
 [94 94 94 ... 94 94 94]
 [94 94 94 ... 94 94 94]
 [94 94 94 ... 94 94 94]]
image: C:\Nikita\Parser\the whole 52 cards deck\TS\0002.png, img: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
image: C:\Nikita\Parser\the whole 52 cards deck\TS\0003.png, img: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
image: C:\Nikita\Parser\the whole 52 cards deck\TS\0004.jpg, img: [[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 

100%|██████████████████████████████████████████████████████████████████████████████████| 52/52 [00:15<00:00,  3.42it/s]

image: C:\Nikita\Parser\the whole 52 cards deck\TS\0017.png, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]]
image: C:\Nikita\Parser\the whole 52 cards deck\TS\0018.jpg, img: [[ 75  77  77 ...  87  89  83]
 [ 77  79  79 ...  84  85  93]
 [ 78  80  80 ...  82  82 100]
 ...
 [ 81  80  79 ...  98 110 101]
 [ 80  81  82 ... 102 109 100]
 [ 79  83  87 ... 105 110 105]]
image: C:\Nikita\Parser\the whole 52 cards deck\TS\0019.jpg, img: [[250 255 252 ... 253 255 251]
 [255 249 249 ... 255 255 247]
 [245 255 255 ... 243 255 251]
 ...
 [248 255 252 ... 237 255 250]
 [255 239 255 ... 255 252 253]
 [253 255 251 ... 252 250 254]]
image: C:\Nikita\Parser\the whole 52 cards deck\TS\0020.jpg, img: [[255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]
 ...
 [255 255 255 ... 255 255 255]
 [255 255 255 ... 255 255 255]


In [119]:
def augment_images(whole_deck_folder, number_of_images, width, height):
    '''
    функция принимает на вход папку whole_deck_folder, содержащую подпапки с изображениями карт, 
    создаёт на том же уровне папку с добавлением к названию "transformed" с подпапками, 
    имеющими те же имена, что в исходной папке, и помещает в подпапки оригинальные и трансформированные изображения, 
    общим числом около number_of_images для каждой игральной карты (подпапки)
    '''
    assert os.path.exists(whole_deck_folder)
    transformed_whole_deck_folder = whole_deck_folder + ' transformed'
    safe_mkdir(transformed_whole_deck_folder)
    
    for cardname in os.listdir(whole_deck_folder):
        card_folder = whole_deck_folder + '\\' + cardname
        folder = transformed_whole_deck_folder + '\\' + cardname
        safe_mkdir(folder)

        for i, image in tqdm(enumerate(os.listdir(card_folder))):
#             srcfile = card_folder + '\\' + image
#             dstfile = folder + '\\' + image
#             shutil.copy2(srcfile, dstfile)
            img = cv2.imread(card_folder + '\\' + image)
            img_resized = cv2.resize(img, (width, height))
#             img_to_write = img_resized.reshape(img_resized.shape[1], img_resized.shape[2], img_resized.shape[3])
            cv2.imwrite(folder + '\\' + image, img_resized)
            img = img.reshape(1, img.shape[0], img.shape[1], img.shape[2])
            data_generator = ImageDataGenerator(rotation_range=90, brightness_range=(0.5, 1.5), shear_range=15.0, zoom_range=[1.5, 1.5])
            data_generator.fit(img)
            image_iterator = data_generator.flow(img)
            X = number_of_images//len(os.listdir(card_folder)) # число аугментированных изображений выбираем таким образом, чтобы в итоге получилось около 1000 изображений каждой карты
            
            for x in range(X):
                img_transformed = image_iterator.next()
                img_transformed = img_transformed.reshape(img_transformed.shape[1], img_transformed.shape[2], img_transformed.shape[3])
                img_transformed = cv2.resize(img_transformed, (width, height))
                ext_len = image.rfind('.')
                filename = image[:ext_len] + str(i) + str(x) + image[ext_len:]
                cv2.imwrite(folder + '\\' + filename, img_transformed)

In [122]:
augment_images(r'C:\Nikita\Parser\the whole 52 cards deck grayscale', 1000, 180, 180)

17it [02:26,  8.64s/it]
18it [05:08, 17.11s/it]
13it [04:48, 22.23s/it]
10it [04:21, 26.13s/it]
11it [03:18, 18.02s/it]
13it [04:07, 19.05s/it]
55it [00:17,  3.16it/s]
9it [08:08, 54.30s/it]
13it [03:22, 15.61s/it]
11it [08:54, 48.55s/it]
12it [09:47, 48.97s/it]
12it [04:22, 21.91s/it]
12it [03:23, 16.92s/it]
17it [06:36, 23.30s/it]
14it [07:44, 33.15s/it]
13it [06:27, 29.77s/it]
17it [04:18, 15.19s/it]
16it [04:41, 17.62s/it]
12it [04:15, 21.29s/it]
25it [05:18, 12.72s/it]
20it [04:01, 12.07s/it]
14it [05:50, 25.04s/it]
11it [06:10, 33.69s/it]
26it [03:32,  8.17s/it]
20it [02:52,  8.64s/it]
63it [00:18,  3.44it/s]
12it [04:14, 21.24s/it]
25it [03:39,  8.79s/it]
18it [02:47,  9.29s/it]
13it [03:12, 14.77s/it]
16it [04:24, 16.50s/it]
23it [04:00, 10.44s/it]
39it [04:40,  7.20s/it]
29it [04:09,  8.59s/it]
35it [04:18,  7.38s/it]
63it [00:25,  2.43it/s]
18it [05:41, 18.98s/it]
23it [05:10, 13.49s/it]
22it [05:19, 14.52s/it]
48it [04:15,  5.33s/it]
30it [04:13,  8.45s/it]
21it [04:14, 12.1

In [126]:
dataset = []

folders = os.listdir('C:/Nikita/Parser/the whole 52 cards deck grayscale transformed')

for folder in folders:
    
    data=[]
    filepath = 'C:/Nikita/Parser/the whole 52 cards deck grayscale transformed/' + folder
    print(filepath)
    assert os.path.exists(filepath)

    for img in tqdm(os.listdir(filepath)):

        label=folder
        img = cv2.imread(filepath+'/'+img, cv2.IMREAD_GRAYSCALE)
        # img = cv2.imread(filepath+'/'+img)
        # img = cv2.resize(img, (180, 180))
        imgs = img.reshape((1, img.shape[0], img.shape[1], 1))
        data.append([imgs, label])
    shuffle(data)

    dataset += data

np.save('C:/Nikita/Parser/data.npy', dataset)

 45%|██████████████████████████████████▋                                          | 452/1003 [00:00<00:00, 2171.45it/s]

C:/Nikita/Parser/the whole 52 cards deck grayscale transformed/2C


  6%|█████                                                                          | 64/1008 [00:00<00:02, 316.99it/s]

C:/Nikita/Parser/the whole 52 cards deck grayscale transformed/2D


  3%|██▌                                                                            | 33/1001 [00:00<00:02, 329.19it/s]

...

 42%|████████████████████████████████▏                                            | 418/1001 [00:00<00:00, 2067.06it/s]

C:/Nikita/Parser/the whole 52 cards deck grayscale transformed/TH


 15%|███████████▌                                                                 | 152/1008 [00:00<00:00, 1493.05it/s]

C:/Nikita/Parser/the whole 52 cards deck grayscale transformed/TS


100%|█████████████████████████████████████████████████████████████████████████████| 1008/1008 [00:02<00:00, 494.85it/s]
C:\Users\administrator\anaconda3\lib\site-packages\numpy\core\_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)


In [4]:
assert 'GPU' in tf.test.gpu_device_name() #run to make sure tensorflow is connected to gpu

In [5]:
data = np.load('C:/Nikita/Parser/data.npy', allow_pickle=True)

In [6]:
x = data[:, 0]
y = data[:, 1]

In [7]:
len(np.unique(y))

52

In [8]:
classes = np.unique(y)
numbers = list(range(len(classes)))
replacement_dict = dict(zip(classes, numbers))

for key in replacement_dict.keys():
    np.place(y, y == key, replacement_dict[key])
    
y = to_categorical(y)

In [9]:
x_new = []
for i in range (len(x)):
    x_new.append(np.array(x[i][0]))
    
x = np.array(x_new)
x.shape

(52569, 180, 180, 1)

In [10]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, shuffle = True, random_state = 42)

In [11]:
for item in (x_train, y_train, x_test, y_test):
    print(item.shape)

(42055, 180, 180, 1)
(42055, 52)
(10514, 180, 180, 1)
(10514, 52)


In [12]:
tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

epochs=25
batch_size=32

model = tf.keras.models.Sequential()

In [ ]:
# plt.imshow(x_train[2].reshape(180,180), cmap = 'Greys')

In [13]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(180, 180, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dropout(0.5),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(52, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 178, 178, 64)      640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 89, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 87, 87, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 20, 20, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 128)      

In [15]:
cp = tf.keras.callbacks.ModelCheckpoint(filepath="C:/Nikita/Cards recognition/25epochs_conv.h5",
                               save_best_only=True,
                               verbose=0)

# model.compile(loss = 'sparse_categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
model.compile(loss = 'categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

In [25]:
history = model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), callbacks=[cp]).history

Epoch 1/25
1315/1315 [==============================] - 51s 39ms/step - loss: 0.6122 - accuracy: 0.8877 - val_loss: 0.4483 - val_accuracy: 0.9148
Epoch 2/25
1315/1315 [==============================] - 51s 39ms/step - loss: 0.5973 - accuracy: 0.8852 - val_loss: 0.6890 - val_accuracy: 0.8917
Epoch 3/25
1315/1315 [==============================] - 52s 39ms/step - loss: 1.1670 - accuracy: 0.8820 - val_loss: 0.4331 - val_accuracy: 0.9425
Epoch 4/25
1315/1315 [==============================] - 52s 39ms/step - loss: 0.6747 - accuracy: 0.8856 - val_loss: 0.5108 - val_accuracy: 0.9271
Epoch 5/25
1315/1315 [==============================] - 52s 39ms/step - loss: 0.7195 - accuracy: 0.8728 - val_loss: 1.2982 - val_accuracy: 0.8884
Epoch 6/25
1315/1315 [==============================] - 52s 40ms/step - loss: 0.7215 - accuracy: 0.8780 - val_loss: 0.5226 - val_accuracy: 0.9211
Epoch 7/25
1315/1315 [==============================] - 52s 39ms/step - loss: 0.7183 - accuracy: 0.8767 - val_loss: 0.9669 -

In [2]:
from matplotlib import pyplot as plt
plt.plot(history['accuracy'], label = 'Accuracy')
plt.plot(history['val_accuracy'], label = 'Validation accuracy')
plt.legend()
plt.show()

NameError: name 'history' is not defined

In [ ]:
model.save('C:/Nikita/Cards recognition/model_52cards.h5')

In [24]:
img = cv2.imread('C:/Nikita/Playing cards/12.jpg', cv2.IMREAD_GRAYSCALE)
img = cv2.resize(img, (180, 180))

img.shape

test_card = img.reshape((1, img.shape[0], img.shape[1], 1))

# plt.imshow(test_card.reshape(180,180), cmap = 'Greys')

prediction = model.predict(test_card)
print(np.argmax(prediction), np.max(prediction))

36 0.27808857


In [298]:
prediction

array([[3.0829231e-08, 2.3873031e-07, 2.6982431e-07, 5.5213697e-17,
        9.0021873e-10, 3.7110545e-05, 7.0160513e-09, 1.6511022e-15,
        4.2992385e-04, 3.3545555e-10, 2.1047830e-11, 1.7927897e-13,
        1.1705093e-12, 4.8520275e-07, 3.8004509e-05, 5.6759442e-09,
        1.2944342e-06, 1.0021119e-02, 4.0849794e-05, 7.4397508e-07,
        1.1918572e-05, 1.2327678e-04, 1.3965407e-05, 1.0214587e-04,
        3.2465374e-03, 2.3038100e-02, 5.6448895e-03, 4.1780114e-04,
        6.7028836e-03, 2.4985094e-04, 9.5042764e-05, 4.8185620e-02,
        1.4066274e-08, 3.7417747e-04, 1.0346625e-05, 5.8126468e-03,
        1.8250122e-07, 5.6341948e-04, 2.4508580e-03, 6.5125879e-03,
        9.3335100e-04, 2.9850699e-04, 8.6371052e-01, 2.9067142e-04,
        3.8183671e-06, 1.7612318e-02, 8.4628660e-04, 9.7205426e-04,
        2.5017234e-04, 6.1939951e-05, 8.7869732e-04, 1.5339110e-05]],
      dtype=float32)

In [14]:
replacement_dict

{'2C': 0,
 '2D': 1,
 '2H': 2,
 '2S': 3,
 '3C': 4,
 '3D': 5,
 '3H': 6,
 '3S': 7,
 '4C': 8,
 '4D': 9,
 '4H': 10,
 '4S': 11,
 '5C': 12,
 '5D': 13,
 '5H': 14,
 '5S': 15,
 '6C': 16,
 '6D': 17,
 '6H': 18,
 '6S': 19,
 '7C': 20,
 '7D': 21,
 '7H': 22,
 '7S': 23,
 '8C': 24,
 '8D': 25,
 '8H': 26,
 '8S': 27,
 '9C': 28,
 '9D': 29,
 '9H': 30,
 '9S': 31,
 'AC': 32,
 'AD': 33,
 'AH': 34,
 'AS': 35,
 'JC': 36,
 'JD': 37,
 'JH': 38,
 'JS': 39,
 'KC': 40,
 'KD': 41,
 'KH': 42,
 'KS': 43,
 'QC': 44,
 'QD': 45,
 'QH': 46,
 'QS': 47,
 'TC': 48,
 'TD': 49,
 'TH': 50,
 'TS': 51}